In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.signal import savgol_filter

In [2]:
data_path = '../data/CALC_STRAIGHT/'
files = [f for f in os.listdir(data_path) if f.endswith('.csv')]

In [3]:
print('STRAIGHT SEGMENTS:\t'+ str(len(files)))

STRAIGHT SEGMENTS:	201


In [4]:
X = pd.read_csv('../data/PN chuze/PN ataxie.csv',usecols=[0,1])
X

,File,Ts [s]
0,N-01-1-Char00,0.008
1,A-03-1-Char00,0.008
2,N-04-1-Char00,0.008
3,N-06-1-Char00,0.008
4,N-07-1-Char00,0.008
5,N-740516-Char00,0.017
6,N-535525-Char00,0.017
7,A-736123-Char00,0.017
8,A-715211-Char00,0.017
9,A-766012-Char00,0.017


In [5]:
stepsR = np.empty(0)
stepsL = np.empty(0)
step_freqR = np.empty(0)
step_freqL = np.empty(0)
t_sampling = np.empty(0)
seg_len = np.empty(0)
t_len = np.empty(0)
starts = np.empty(0)
ends = np.empty(0)

for file in files:
    subject =  file.split(sep='Char00',maxsplit=1)[0]+'Char00'
    print(file)
    
    df = pd.read_csv(data_path + file)
    N = np.size(df,axis=0)
    seg_len = np.append(seg_len,N)
    
    T_sampling = X.loc[X['File'] == subject,'Ts [s]'].values
    t_sampling = np.append(t_sampling,T_sampling)
    
    T_len = N*T_sampling
    t_len = np.append(t_len,T_len)
       
    start = df.OrigIndex.iloc[0]
    end = df.OrigIndex.iloc[-1]
    starts = np.append(starts,start)
    ends = np.append(ends,end)
    
    peaksL = np.where(np.diff(df.contactL)>0)[0]
    peaksR = np.where(np.diff(df.contactR)>0)[0]
    stepsL = np.append(stepsL,len(peaksL))
    stepsR = np.append(stepsR,len(peaksR))
    
    freqL = 1 / (T_len / len(peaksL)) 
    freqR = 1 / (T_len / len(peaksR)) 
    step_freqL = np.append(step_freqL,freqL)
    step_freqR = np.append(step_freqR,freqR)

N-04-1-Char00part7seg-3.csv
A-736123-Char00part1seg-0.csv
N-15051994-Char00part2seg-0.csv
N-15051994-Char00part3seg-0.csv
N-920828-Char00part1seg-0.csv
N-04-1-Char00part7seg-2.csv
N-535525-Char00part1seg-8.csv
N-04-1-Char00part7seg-0.csv
N-04-1-Char00part6seg-0.csv
N-04-1-Char00part7seg-1.csv
N-15051994-Char00part8seg-0.csv
N-740516-Char00part5seg-0.csv
N-740516-Char00part4seg-0.csv
N-07-1-Char00part1seg-19.csv
N-25031996-Char00part3seg-10.csv
N-07-1-Char00part1seg-18.csv
N-01-1-Char00part1seg-0.csv
N-740516-Char00part4seg-1.csv
N-25031996-Char00part1seg-0.csv
N-740516-Char00part4seg-3.csv
N-10-1-Char00part5seg-0.csv
N-06-1-Char00part1seg-0.csv
N-10-1-Char00part4seg-0.csv
A-715211-Char00part2seg-0.csv
N-740516-Char00part4seg-2.csv
A-715211-Char00part1seg-8.csv
N-07-1-Char00part1seg-0.csv
N-25031996-Char00part3seg-9.csv
N-740516-Char00part2seg-0.csv
N-740516-Char00part3seg-0.csv
A-615829-Char00part1seg-5.csv
A-09-1-Char00part1seg-3.csv
N-15051994-Char00part4seg-6.csv
A-09-1-Char00part1s

In [6]:
A = pd.DataFrame(data={'File':files, 'IndexStart':starts, 'IndexEnd':ends, 'StepsL':stepsL, 'StepsR':stepsR, 'Ts':t_sampling,
                       'SegLen':seg_len, 'TimeLen':t_len, 'StepFreqL':step_freqL, 'StepFreqR':step_freqR})
for file in A.File: A.loc[A.File==file,'Type'] = file[0]
A.head()

,File,IndexStart,IndexEnd,StepsL,StepsR,Ts,SegLen,TimeLen,StepFreqL,StepFreqR,Type
0,N-04-1-Char00part7seg-3.csv,59744.0,61820.0,14.0,14.0,0.008,2077.0,16.616,0.842561,0.842561,N
1,A-736123-Char00part1seg-0.csv,2900.0,4287.0,18.0,18.0,0.017,1388.0,23.596,0.762841,0.762841,A
2,N-15051994-Char00part2seg-0.csv,18500.0,20250.0,13.0,13.0,0.008,1751.0,14.008,0.928041,0.928041,N
3,N-15051994-Char00part3seg-0.csv,22296.0,23920.0,12.0,12.0,0.008,1625.0,13.000,0.923077,0.923077,N
4,N-920828-Char00part1seg-0.csv,2568.0,3367.0,12.0,11.0,0.017,800.0,13.600,0.882353,0.808824,N


In [7]:
A.to_csv('../data/SEGMENTS.csv',index=False)